In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [2]:
conf = SparkConf()

conf.set("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1")

conf.set('loglevel', 'ERROR')

SparkContext(conf=conf)

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkStreaming") \
    .getOrCreate()

In [3]:
from pyspark.sql.streaming import *

In [5]:
spark

In [6]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.1

Error: Missing application resource.

Usage: spark-submit [options] <app jar | python file | R file> [app arguments]
Usage: spark-submit --kill [submission ID] --master [spark://...]
Usage: spark-submit --status [submission ID] --master [spark://...]
Usage: spark-submit run-example [options] example-class [example args]

Options:
  --master MASTER_URL         spark://host:port, mesos://host:port, yarn,
                              k8s://https://host:port, or local (Default: local[*]).
  --deploy-mode DEPLOY_MODE   Whether to launch the driver program locally ("client") or
                              on one of the worker machines inside the cluster ("cluster")
                              (Default: client).
  --class CLASS_NAME          Your application's main class (for Java / Scala apps).
  --name NAME                 A name of your application.
  --jars JARS                 Comma-separated list of jars to include on the driver
                              and executor classpaths

In [4]:
from pyspark.sql.types import StructType
from pyspark.sql import Row
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import *

In [5]:
kafkaDF = spark\
        .readStream.format("kafka") \
        .option("kafka.bootstrap.servers","kafka:9093") \
        .option("subscribe","velib-stations") \
        .load()

In [6]:
kafkaDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
#on crée un dataframe qui represente le flux en entrée de kafka

KAFKA_SERVER = "localhost:9093"
KAFKA_TOPIC = "velib-stations"


schema = StructType([
    StructField("contract_name", StringType(), True),
    #StructField("last_update", TimestampType(), False),
    StructField("number", IntegerType(), True),
    StructField("address", StringType(), True),
    StructField("status", StringType(), True),
    StructField("bike_stands", IntegerType(), True),
])
    
kafkaDF = spark\
        .readStream.format("kafka") \
        .option("kafka.bootstrap.servers","kafka:9092") \
        .option("subscribe","velib-stations") \
        .load()\
        .select(from_json(col("value").cast("string"), schema).alias("parsed_value")) \
        .select(col("parsed_value.*"))

In [8]:
print(type(kafkaDF))

<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
kafkaDF.printSchema()

root
 |-- contract_name: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- status: string (nullable = true)
 |-- bike_stands: integer (nullable = true)



In [10]:
#raw = spark.sql("select * from qraw")

In [ ]:
query = kafkaDF.writeStream \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .outputMode("append") \
    .start()

query.awaitTermination()